In [1]:
import os
from dotenv import load_dotenv

load_dotenv("./credentials.env")

True

# [そろそろ知っておかないとヤバい？ 話題のLangChainを30分だけ触って理解しよう！](https://qiita.com/minorun365/items/081fc560e08f0197a7a8)

In [2]:
# Chat model
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=os.getenv("OPENAI_MODEL_NAME"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0,
    model_kwargs={
        "deployment_id": os.getenv("OPENAI_DEPLOYMENT_ID"),
        "api_type": os.getenv("OPENAI_API_TYPE"),
        "api_version": os.getenv("OPENAI_API_VERSION"),
    },
)

In [3]:
# Schema
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

input = [
    HumanMessage(content="日本の総理大臣は誰ですか？"),
]

llm.predict_messages(input)

AIMessage(content='2022年10月現在、日本の総理大臣は岸田文雄（きしだ ふみお）です。')

In [4]:
# Prompt template
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("{keyword} を解説するブログタイトル案は?")
prompt = template.format(keyword="Python の基礎")

llm.predict(prompt)

'「Pythonの基礎をわかりやすく解説！初心者向けブログ」\n「Python入門者必見！基礎から学ぶPython解説ブログ」\n「Pythonの基礎をマスターしよう！解説ブログ」\n「Python初心者のための基礎解説ブログ」\n「Pythonの基礎を丁寧に解説！初心者向けブログ」'

In [5]:
prompt = template.format(keyword="Azure の基礎")
llm.predict(prompt)

'「Azureの基礎をわかりやすく解説！初心者向けブログ」\n「Azure入門！基礎知識から解説するブログ」\n「Azureの基礎を徹底解説！初心者でもわかるブログ」\n「Azureの基本を学ぶならこのブログ！初心者向け解説」\n「Azureの基礎知識を網羅的に解説するブログ」'

In [6]:
# Chain
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=template)
chain.run("AWS")

'「AWSとは？クラウドコンピューティングの基礎を解説！」\n「AWSの魅力とは？ビジネスにおける活用方法を解説！」\n「AWSの主要サービスを徹底解説！初心者でもわかりやすく紹介します」\n「AWSの利点とは？クラウドサービスのメリットを解説！」\n「AWSのセキュリティ対策について解説！安心して利用するためのポイント」'

In [7]:
# Agent
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("144 の平方根はいくつ？")



> Entering new AgentExecutor chain...
I need to find the square root of 144.
Action: Calculator
Action Input: √144
Observation: Answer: 12.0
Thought:I now know the final answer
Final Answer: The square root of 144 is 12.

> Finished chain.


'The square root of 144 is 12.'

In [8]:
# Memory
from langchain import ConversationChain

conversation = ConversationChain(llm=llm, verbose=True)

conversation.run("私は日本人です。何を聞いても日本語で答えてください。")
conversation.run("Who is the prime minister of Japan?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 私は日本人です。何を聞いても日本語で答えてください。
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 私は日本人です。何を聞いても日本語で答えてください。
AI: こんにちは！日本語でお答えします。どのような質問がありますか？
Human: Who is the prime minister of Japan?
AI:

> Finished chain.


'現在の日本の首相は菅義偉（すが・よしひで）です。彼は2020年9月に就任しました。'

[これを読んだらわかった気になれるLangChain
](https://qiita.com/moritalous/items/7ceccbf809cc2f35ded2)

In [9]:
# Chat Prompt templates
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "あなたは{input_language}から{output_language}に翻訳するのに役立つアシスタントです。"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt,
])

# get a chat completion from the formatted messages
llm.predict_messages(chat_prompt.format_prompt(
    input_language="Japanese",
    output_language="English",
    text="私はプログラミングが大好きです。").to_messages())

AIMessage(content='I love programming.')

In [10]:
llm.predict_messages(chat_prompt.format_prompt(
    input_language="Japanese",
    output_language="French",
    text="私はプログラミングが大好きです。").to_messages())

AIMessage(content="J'adore la programmation.")